# Simple Perception Stack For Self-Driving Cars

![Logo](./logo.png)

**Kareim Tarek AbdelAzeem Amin         1701002**

https://github.com/KareimGazer/Perception-Stack

# Abstract
In this first part of the project we aim to detect and draw the lane line. the frames of the video pass through a pipleline to get processed and achieve the required results. In the following sections we demonstrate those steps.

# Methodology
to achieve that the video frame pass through the following steps of the pipeline

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

# prototype
the following snippets of code demonstrate the pipleline on images. for videos please run `run.sh` in terminal or see the last two code cells.

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2
%matplotlib inline

## Camera Calibration
Cameras needed to be calibrated first to remove distortion. we import `calibrate_camera` module and compute correction matrix and distortion coefficients on provided chessboard images in camera_cal folder.

In [ ]:
import calibrate_camera # calibration module
mtx, dist = calibrate_camera.calibrate(9, 6, 'camera_cal/*.jpg')

In [ ]:
mtx, dist

after we got the matrix and distortion coefficients we apply them to checkboards images to check the results.
results can be found at `output_images/calibrated_boards/`, and a sample is plotted inline

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
images_names = glob.glob('camera_cal/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    undistored_image = calibrate_camera.undistort(image, mtx, dist)
    cv2.imwrite('output_images/calibrated_boards/test{}.jpg'.format(index), undistored_image)

cal_board = plt.imread('output_images/calibrated_boards/test0.jpg')
plt.imshow(cal_board)

we save the results in a pickle dictionary so we don't need to repeat the pipeline and gain performance improvements

In [ ]:
import pickle
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )

## Undistort
Now we try to correct the road Images.

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
images_names = glob.glob('test_images/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    undistorted_image = calibrate_camera.undistort(image, mtx, dist)
    cv2.imwrite('output_images/calibrated_roads/road{}.jpg'.format(index), undistorted_image)

undist_road = plt.imread('output_images/calibrated_roads/road0.jpg')
plt.imshow(undist_road)

## Use color transforms, gradients, etc., to create a thresholded binary image.

I used combination of the B channel In LAB color representation anded with S channel in HLS representation to detect the left yellow lane, and the L channel in the HLS color representation to detect white right lanes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import sobel # sobel module reads images in RGB using matplot lib
images_names = glob.glob('output_images/calibrated_roads/*.jpg')
ksize = 3
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    combined_binary = sobel.get_binary(image, ksize)
    plt.imsave('output_images/roads_binary/{}.jpg'.format(index), combined_binary,
               cmap='gray')
thresh_road = plt.imread('output_images/roads_binary/0.jpg')
plt.imshow(thresh_road)

## Apply a perspective transform to rectify binary image ("birds-eye view").

the code is in `bird_view.py` file and I used these points [[530, 490], [750, 490], [210, 690], [1080, 690]]
it only contains one function responsible for the transformation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import bird_view
import lanes
import cv2
import rad

images_names = glob.glob('output_images/roads_binary/*.jpg')
# undists = glob.glob('output_images/calibrated_roads/*.jpg')
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(image)
    plt.imsave('output_images/roads_view/{}.jpg'.format(index), binary_warped,
               cmap='gray')
thresh_road = plt.imread('output_images/roads_view/0.jpg')
plt.imshow(thresh_road)

## Detect lane pixels and fit to find the lane boundary.
I used the sliding window methodolgy as follows:
- get the indicies of the lanes using histogram to detect the place of max intensities
- create a window and if the number of pixels in it is large get the mean of these pixels and move the window accordingly
- repeat the steps until the frame is full of windows

the minimum number of pixels, window size, and number of windows are all hyper parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import bird_view
import lanes
import cv2
import rad

images_names = glob.glob('output_images/roads_view/*.jpg')
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(image_gray)
    plt.imsave('output_images/roads_plt/{}.jpg'.format(index), out_img)
plt_road = plt.imread('output_images/roads_plt/0.jpg')
plt.imshow(plt_road)

## Output visual display of the lane boundaries
after extracting warped binary image and fitting a polynomical to the pixels detected by the previous windows, we use these lines to draw and shade the space between the lanes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import bird_view
import lanes
import cv2
import rad

images_names = glob.glob('output_images/roads_binary/*.jpg')
undists = glob.glob('output_images/calibrated_roads/*.jpg')

for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(image_gray)
    out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped)
    
    undist = plt.imread(undists[index])
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, undist)
    
    left_curverad, right_curverad, offset = rad.measure_curvature_real(binary_warped, left_fitx, right_fitx, ploty)
    
    print("{}: curvature ({} m, {} m), offset = {} m".format(index, left_curverad, right_curverad, offset))
    plt.imsave('output_images/roads_maped/{}.jpg'.format(index), result)
plt_road = plt.imread('output_images/roads_maped/0.jpg')
plt.imshow(plt_road)

## pipeline prototype
the pipeline in one function

In [ ]:
import bird_view
import lanes
import rad
import sobel

prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = (None, None, None, None)

def pipeline(frame, mtx, dist):
    global prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty
    undistored_image = calibrate_camera.undistort(frame, mtx, dist)
    # image_rgb = cv2.cvtColor(undistored_image, cv2.COLOR_BGR2RGB)
    combined_soble = sobel.get_binary(undistored_image, ksize)
    # return combined_soble

    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(combined_soble)
    
    out_img, left_fitx, right_fitx, ploty = (None, None, None, None)
    try:
        out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped, 10, 90, 50)
        prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = out_img, left_fitx, right_fitx, ploty
    except:
        out_img, left_fitx, right_fitx, ploty = prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty
    
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, frame)
    
    # calculating curvature and center offset
    left_curverad, right_curverad, real_offset = rad.measure_curvature_real(binary_warped, left_fitx, right_fitx, ploty)
    curve_info = "radius of curvature ({} Km, {} Km)".format(str(round(left_curverad/1000, 2)), 
                                                           str(round(right_curverad/1000, 2)))
    
    center_info = "offset from center  = {} m".format(str(round(real_offset, 2)))
    
    detailed = cv2.putText(result, curve_info, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    detailed = cv2.putText(detailed, center_info, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    return result

## Live Processing

In [ ]:
import cv2
import numpy as np
import sobel
import calibrate_camera

# Creating a VideoCapture object to read the video
# cap = cv2.VideoCapture('project_video.mp4')
cap = cv2.VideoCapture('challenge_video.mp4')
# cap = cv2.VideoCapture('harder_challenge_video.mp4') 

ksize = 3

# Loop until the end of the video
while (cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720), fx = 0, fy = 0,
                         interpolation = cv2.INTER_CUBIC)
 
    # Display the resulting frame
    cv2.imshow('Frame', frame)
    result = pipeline(frame, mtx, dist)
    cv2.imshow('gblur', result)

 
    # define q as the exit button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# release the video capture object
cap.release()

# Closes all the windows currently opened.
cv2.destroyAllWindows()

## store video

In [ ]:
import numpy as np
import cv2

ksize = 3

# cap = cv2.VideoCapture('challenge_video.mp4')
cap = cv2.VideoCapture('project_video.mp4')

video_file = 'race1.mp4'
frame_size = (1280, 720)
fps = 40
out = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'MP4V'), fps, frame_size)

# Loop until the end of the video
while (cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.resize(frame, frame_size, fx = 0, fy = 0,
                         interpolation = cv2.INTER_CUBIC)
 
    # Display the resulting frame
    # cv2.imshow('Frame', frame)
    result = pipeline(frame, mtx, dist)
    cv2.imshow('image', result)
    # result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    out.write(result)

    # define q as the exit button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the video capture object
cap.release()
out.release()
# Closes all the windows currently opened.
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

def out_vid():
    ksize = 3

    # cap = cv2.VideoCapture('challenge_video.mp4')
    cap = cv2.VideoCapture('project_video.mp4')

    video_file = 'race1.mp4'
    frame_size = (1280, 720)
    fps = 40
    out = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'MP4V'), fps, frame_size)

    # Loop until the end of the video
    while (cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        frame = cv2.resize(frame, frame_size, fx = 0, fy = 0,
                         interpolation = cv2.INTER_CUBIC)
 
        # Display the resulting frame
        # cv2.imshow('Frame', frame)
        result = pipeline(frame, mtx, dist)
        cv2.imshow('image', result)
        result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
        out.write(result)

        # define q as the exit button
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # release the video capture object
    cap.release()
    out.release()
    # Closes all the windows currently opened.
    cv2.destroyAllWindows()

In [ ]:
%time out_vid()

# Discussion

The pipeline could benefit from mergeing multiple detection techniques like anding the color channels with the gradients obtained from sobel for example to get robust lines. also the use of adaptive thresholding, and contrast equilization could improve the performance, though I didn't much difference from applying them.

## use videpy

In [ ]:
import calibrate_camera # calibration module
mtx, dist = calibrate_camera.calibrate(9, 6, 'camera_cal/*.jpg')
ksize = 3

import numpy as np
import cv2
import bird_view
import lanes
import rad
import sobel

prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = (None, None, None, None)

def detect_lanes(frame):
    global prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty, mtx, dist, ksize
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    undistored_image = calibrate_camera.undistort(frame, mtx, dist)
    # image_rgb = cv2.cvtColor(undistored_image, cv2.COLOR_BGR2RGB)
    combined_soble = sobel.get_binary(undistored_image, ksize)
    # return combined_soble

    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(combined_soble)
    
    out_img, left_fitx, right_fitx, ploty = (None, None, None, None)
    try:
        out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped, 10, 90, 50)
        prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = out_img, left_fitx, right_fitx, ploty
    except:
        out_img, left_fitx, right_fitx, ploty = prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty
    
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, frame)
    
    # calculating curvature and center offset
    left_curverad, right_curverad, real_offset = rad.measure_curvature_real(binary_warped, left_fitx, right_fitx, ploty)
    curve_info = "radius of curvature ({} Km, {} Km)".format(str(round(left_curverad/1000, 2)), 
                                                           str(round(right_curverad/1000, 2)))
    
    center_info = "offset from center  = {} m".format(str(round(real_offset, 2)))
    
    detailed = cv2.putText(result, curve_info, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    detailed = cv2.putText(detailed, center_info, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    return cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

In [ ]:
from moviepy.editor import VideoFileClip

# project_video_path = 'project_video.mp4'
project_video_path = 'challenge_video.mp4'
project_video_output = 'output.mp4'

In [ ]:
project_video = VideoFileClip(project_video_path)
out_clip = project_video.fl_image(detect_lanes) #NOTE: this function expects color images!!
%time out_clip.write_videofile(project_video_output, audio=False)

## debug lanes

In [16]:
import calibrate_camera # calibration module
mtx, dist = calibrate_camera.calibrate(9, 6, 'camera_cal/*.jpg')
ksize = 3

import numpy as np
import cv2
import bird_view
import lanes
import rad
import sobel

prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = (None, None, None, None)

def debug_pipeline(frame, mtx, dist):
    global prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty
    undistored_image = calibrate_camera.undistort(frame, mtx, dist)
    combined_soble = sobel.get_binary(undistored_image, ksize)
    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(combined_soble)
    out_img, left_fitx, right_fitx, ploty = (None, None, None, None)
    try:
        out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped, 10, 90, 50)
        # print("out_img", out_img.shape)
        prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty = out_img, left_fitx, right_fitx, ploty
    except:
        out_img, left_fitx, right_fitx, ploty = prev_out_img, prev_left_fitx, prev_right_fitx, prev_ploty
    
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, frame)
    left_curverad, right_curverad, real_offset = rad.measure_curvature_real(binary_warped, left_fitx, right_fitx, ploty)
    curve_info = "radius of curvature ({} Km, {} Km)".format(str(round(left_curverad/1000, 2)), 
                                                           str(round(right_curverad/1000, 2)))
    center_info = "offset from center  = {} m".format(str(round(real_offset, 2)))
    detailed = cv2.putText(result, curve_info, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    detailed = cv2.putText(detailed, center_info, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 0), 2, cv2.LINE_AA)
    combined_soble = np.dstack((combined_soble, combined_soble, combined_soble))
    binary_warped = np.dstack((binary_warped, binary_warped, binary_warped))
    return undistored_image, combined_soble, binary_warped, out_img, detailed

def get_debug_image(frame):
    global mtx, dist
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    undistored_image, combined_soble, binary_warped, out_img, detailed = debug_pipeline(frame, mtx, dist)
    
    undistored_image = cv2.cvtColor(undistored_image, cv2.COLOR_BGR2RGB)
    detailed = cv2.cvtColor(detailed, cv2.COLOR_BGR2RGB)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    combined_soble = cv2.resize(combined_soble, (0, 0), None, .5, .5)
    undistored_image = cv2.resize(undistored_image, (0, 0), None, .5, .5)
    binary_warped = cv2.resize(binary_warped, (0, 0), None, .25, .5)
    out_img = cv2.resize(out_img, (0, 0), None, .25, .5)
    detailed = cv2.resize(detailed, (0, 0), None, .25, .5)
    frame =  cv2.resize(frame, (0, 0), None, .25, .5)
    
    numpy_horz1 = np.hstack((undistored_image, combined_soble*255)) # x * 2
    numpy_horz2 = np.hstack((frame, binary_warped*255, out_img, detailed))
    numpy_ver = np.vstack((numpy_horz1, numpy_horz2))
    return numpy_ver

In [17]:
from moviepy.editor import VideoFileClip

# project_video_path = 'project_video.mp4'
project_video_path = 'challenge_video.mp4'
project_video_output = 'output.mp4'

In [18]:
project_video = VideoFileClip(project_video_path)
out_clip = project_video.fl_image(get_debug_image) #NOTE: this function expects color images!!
%time out_clip.write_videofile(project_video_output, audio=False)

                                                                                                                       

                                                                                                                 
t:  27%|█████████████████▊                                                 | 129/485 [07:53<00:45,  7.90it/s, now=None]

t:  14%|█████████▊                                                          | 70/485 [03:35<00:57,  7.18it/s, now=None]
                                                                                                                       

                                                                                                                 
t:  27%|█████████████████▊                                                 | 129/485 [07:53<00:45,  7.90it/s, now=None]

t:  35%|███████████████████████▏                                           | 168/485 [05:13<00:39,  7.96it/s, now=None]

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4






t:   0%|                                                                             | 0/485 [00:00<?, ?it/s, now=None]


t:   0%|▎                                                                    | 2/485 [00:00<00:39, 12.15it/s, now=None]


t:   1%|▌                                                                    | 4/485 [00:00<00:56,  8.45it/s, now=None]


t:   1%|▋                                                                    | 5/485 [00:00<00:56,  8.44it/s, now=None]


t:   1%|▊                                                                    | 6/485 [00:00<00:58,  8.21it/s, now=None]


t:   1%|▉                                                                    | 7/485 [00:00<01:01,  7.78it/s, now=None]


t:   2%|█▏                                                                   | 8/485 [00:00<00:57,  8.24it/s, now=None]


t:   2%|█▎                                                                   | 9/485 [00:01<00:57,  8.28it/s, now=None]


t:   2%|█▍           

t:  14%|█████████▊                                                          | 70/485 [00:08<00:57,  7.18it/s, now=None]


t:  15%|█████████▉                                                          | 71/485 [00:08<00:57,  7.23it/s, now=None]


t:  15%|██████████                                                          | 72/485 [00:09<00:58,  7.09it/s, now=None]


t:  15%|██████████▏                                                         | 73/485 [00:09<00:55,  7.46it/s, now=None]


t:  15%|██████████▍                                                         | 74/485 [00:09<00:53,  7.73it/s, now=None]


t:  16%|██████████▋                                                         | 76/485 [00:09<00:48,  8.51it/s, now=None]


t:  16%|██████████▊                                                         | 77/485 [00:09<00:50,  8.13it/s, now=None]


t:  16%|██████████▉                                                         | 78/485 [00:09<00:51,  7.85it/s, now=None]


t:  16%|███████████     

t:  28%|██████████████████▌                                                | 134/485 [00:16<00:43,  8.03it/s, now=None]


t:  28%|██████████████████▋                                                | 135/485 [00:17<00:42,  8.19it/s, now=None]


t:  28%|██████████████████▊                                                | 136/485 [00:17<00:41,  8.33it/s, now=None]


t:  28%|██████████████████▉                                                | 137/485 [00:17<00:43,  8.09it/s, now=None]


t:  28%|███████████████████                                                | 138/485 [00:17<00:45,  7.62it/s, now=None]


t:  29%|███████████████████▏                                               | 139/485 [00:17<00:44,  7.75it/s, now=None]


t:  29%|███████████████████▎                                               | 140/485 [00:17<00:44,  7.73it/s, now=None]


t:  29%|███████████████████▍                                               | 141/485 [00:17<00:45,  7.55it/s, now=None]


t:  29%|████████████████

...

t:  54%|████████████████████████████████████                               | 261/485 [00:34<00:31,  7.13it/s, now=None]


t:  54%|████████████████████████████████████▏                              | 262/485 [00:34<00:29,  7.47it/s, now=None]


t:  54%|████████████████████████████████████▎                              | 263/485 [00:34<00:27,  8.02it/s, now=None]


t:  54%|████████████████████████████████████▍                              | 264/485 [00:34<00:27,  7.94it/s, now=None]


t:  55%|████████████████████████████████████▌                              | 265/485 [00:34<00:29,  7.55it/s, now=None]


t:  55%|████████████████████████████████████▋                              | 266/485 [00:34<00:29,  7.48it/s, now=None]


t:  55%|████████████████████████████████████▉                              | 267/485 [00:34<00:28,  7.52it/s, now=None]


t:  55%|█████████████████████████████████████                              | 268/485 [00:35<00:31,  6.98it/s, now=None]


t:  55%|████████████████

t:  67%|████████████████████████████████████████████▌                      | 323/485 [00:42<00:21,  7.46it/s, now=None]


t:  67%|████████████████████████████████████████████▊                      | 324/485 [00:42<00:23,  6.97it/s, now=None]


t:  67%|████████████████████████████████████████████▉                      | 325/485 [00:42<00:22,  7.25it/s, now=None]


t:  67%|█████████████████████████████████████████████                      | 326/485 [00:43<00:20,  7.69it/s, now=None]


t:  67%|█████████████████████████████████████████████▏                     | 327/485 [00:43<00:19,  8.12it/s, now=None]


t:  68%|█████████████████████████████████████████████▎                     | 328/485 [00:43<00:18,  8.27it/s, now=None]


t:  68%|█████████████████████████████████████████████▌                     | 330/485 [00:43<00:16,  9.18it/s, now=None]


t:  68%|█████████████████████████████████████████████▋                     | 331/485 [00:43<00:18,  8.32it/s, now=None]


t:  68%|████████████████

t:  80%|█████████████████████████████████████████████████████▍             | 387/485 [00:51<00:14,  6.96it/s, now=None]


t:  80%|█████████████████████████████████████████████████████▌             | 388/485 [00:51<00:14,  6.67it/s, now=None]


t:  80%|█████████████████████████████████████████████████████▋             | 389/485 [00:51<00:14,  6.78it/s, now=None]


t:  80%|█████████████████████████████████████████████████████▉             | 390/485 [00:51<00:13,  7.22it/s, now=None]


t:  81%|██████████████████████████████████████████████████████             | 391/485 [00:51<00:12,  7.62it/s, now=None]


t:  81%|██████████████████████████████████████████████████████▏            | 392/485 [00:51<00:12,  7.65it/s, now=None]


t:  81%|██████████████████████████████████████████████████████▎            | 393/485 [00:52<00:11,  7.92it/s, now=None]


t:  81%|██████████████████████████████████████████████████████▍            | 394/485 [00:52<00:11,  8.24it/s, now=None]


t:  81%|████████████████

t:  93%|██████████████████████████████████████████████████████████████▏    | 450/485 [00:59<00:04,  8.30it/s, now=None]


t:  93%|██████████████████████████████████████████████████████████████▎    | 451/485 [00:59<00:04,  7.97it/s, now=None]


t:  93%|██████████████████████████████████████████████████████████████▍    | 452/485 [00:59<00:04,  7.83it/s, now=None]


t:  93%|██████████████████████████████████████████████████████████████▌    | 453/485 [01:00<00:04,  7.77it/s, now=None]


t:  94%|██████████████████████████████████████████████████████████████▋    | 454/485 [01:00<00:04,  7.46it/s, now=None]


t:  94%|██████████████████████████████████████████████████████████████▊    | 455/485 [01:00<00:04,  7.10it/s, now=None]


t:  94%|██████████████████████████████████████████████████████████████▉    | 456/485 [01:00<00:03,  7.27it/s, now=None]


t:  94%|███████████████████████████████████████████████████████████████▎   | 458/485 [01:00<00:03,  8.66it/s, now=None]


t:  95%|████████████████

Moviepy - Done !
Moviepy - video ready output.mp4
CPU times: total: 2min 20s
Wall time: 1min 6s
